In [ ]:
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df = pd.read_csv("../../image_pairs_mos.csv", index_col=0)
print(df.shape)
df.head()

(1700, 6)


,Reference,Distorted,MOS,Reference_ID,Distortion_ID,Distortion_Intensity
0,I01.BMP,I01_01_1.bmp,5.9706,1,1,1
1,I01.BMP,I01_01_2.bmp,5.4167,1,1,2
2,I01.BMP,I01_01_3.bmp,4.5556,1,1,3
3,I01.BMP,I01_01_4.bmp,4.3143,1,1,4
4,I01.BMP,I01_02_1.bmp,6.1429,1,2,1


The idea is that we can generate a contrastive dataset if we take combinations of all the transformations and create triplets with them. Then, we will be able to use their respective MOS to say which should be closer to the reference image.

In [ ]:
from itertools import combinations

In [ ]:
df[df.Reference_ID == 1]

,Reference,Distorted,MOS,Reference_ID,Distortion_ID,Distortion_Intensity
0,I01.BMP,I01_01_1.bmp,5.9706,1,1,1
1,I01.BMP,I01_01_2.bmp,5.4167,1,1,2
2,I01.BMP,I01_01_3.bmp,4.5556,1,1,3
3,I01.BMP,I01_01_4.bmp,4.3143,1,1,4
4,I01.BMP,I01_02_1.bmp,6.1429,1,2,1
...,...,...,...,...,...,...
63,I01.BMP,I01_16_4.bmp,4.8571,1,16,4
64,I01.BMP,I01_17_1.bmp,6.5000,1,17,1
65,I01.BMP,I01_17_2.bmp,4.9714,1,17,2
66,I01.BMP,I01_17_3.bmp,6.1944,1,17,3


In [ ]:
idxs = list(combinations(df[df.Reference_ID == 1].index, r=2))
len(idxs)

2278

In [ ]:
rows = []
for ref_id in tqdm(pd.unique(df.Reference_ID)):
    for index in combinations(df[df.Reference_ID == ref_id].index, r=2):
        first = df.loc[index,:].iloc[0:1]
        second = df.loc[index,:].iloc[1:2]
        one_row = first.join(second, lsuffix="_0", rsuffix="_1", how="cross")
        one_row = one_row.drop("Reference_1", axis=1)
        one_row = one_row[["Reference_0", "Distorted_0", "Distorted_1", "MOS_0", "MOS_1", "Reference_ID_0", "Distortion_ID_0", "Distortion_Intensity_0", "Reference_ID_1", "Distortion_ID_1", "Distortion_Intensity_1"]]
        one_row = one_row.rename({"Reference_0": "Reference"}, axis=1)
        rows.append(one_row)
df_comb = pd.concat(rows)

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
print(df_comb.shape)
df_comb.head()

(56950, 11)


,Reference,Distorted_0,Distorted_1,MOS_0,MOS_1,Reference_ID_0,Distortion_ID_0,Distortion_Intensity_0,Reference_ID_1,Distortion_ID_1,Distortion_Intensity_1
0,I01.BMP,I01_01_1.bmp,I01_01_2.bmp,5.9706,5.4167,1,1,1,1,1,2
0,I01.BMP,I01_01_1.bmp,I01_01_3.bmp,5.9706,4.5556,1,1,1,1,1,3
0,I01.BMP,I01_01_1.bmp,I01_01_4.bmp,5.9706,4.3143,1,1,1,1,1,4
0,I01.BMP,I01_01_1.bmp,I01_02_1.bmp,5.9706,6.1429,1,1,1,1,2,1
0,I01.BMP,I01_01_1.bmp,I01_02_2.bmp,5.9706,5.7941,1,1,1,1,2,2


In [ ]:
df_comb.to_csv("image_pairs_mos_contrastive.csv", index=False)